In [148]:
%pip install pip install -q -U google-genai
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Isaác Mauricio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for install


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Isaác Mauricio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Isaác Mauricio\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [149]:
# Importar librerías correspondientes
# Google Gemini API
from typing import Optional, Literal
from pydantic import BaseModel, Field
from google import genai
from google.genai import types

# Operaciones de sistema
import os
from dotenv import load_dotenv
import logging
import datetime

# Gmail API
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [150]:

# Configurar los logs
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)

# Cargamos nuestra API_KEY de Gemini
load_dotenv()
API_KEY = os.getenv("API_KEY")
# Se define la variable global del modelo a utilizar en las requests
model="gemini-2.0-flash"


In [151]:
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

def get_calendar_services():
    """
    Obtiene el servicio de la API de Google Calendar autorizado.
    """
    creds = None
    # El archivo token.json guarda los tokens de acceso y refresca del usuario,
    # y se crea automáticamente la primera vez que se ejecuta el flujo de autorización.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # Si no hay credenciales disponibles o no son válidas, se pide al usuario que se autorice.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            from google_auth_oauthlib.flow import InstalledAppFlow
            flow = InstalledAppFlow.from_client_secrets_file(
                'gmail_cred.json', SCOPES) # Reemplaza 'credentials.json' con el nombre de tu archivo de credenciales
            creds = flow.run_local_server(port=0)
        # Guarda las credenciales para la próxima ejecución
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('calendar', 'v3', credentials=creds)
        return service
    except HttpError as error:
        print(f'Ocurrió un error al construir el servicio de calendario: {error}')
        return None


def get_calendar_events(service, days=7, calendar_id='primary'):
    """
    Obtiene los eventos futuros del calendario.

    Args:
        service: El servicio de la API de Google Calendar autorizado.
        dias_en_el_futuro: El número de días en el futuro para buscar eventos.
        calendar_id: El ID del calendario del que se quieren obtener los eventos. Por defecto, es 'primary' (calendario principal del usuario).

    Returns:
        Una lista de eventos futuros, donde cada evento es un diccionario con información sobre el mismo.
        Devuelve una lista vacía si no se encuentran eventos o si ocurre un error.
    """
    now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indica hora UTC
    timeMin = datetime.datetime.utcnow() + datetime.timedelta(days=0)  # Ajusta aquí si quieres empezar desde mañana
    timeMax = datetime.datetime.utcnow() + datetime.timedelta(days=days)
    timeMin_iso = timeMin.isoformat() + 'Z'
    timeMax_iso = timeMax.isoformat() + 'Z'


    logger.info(f'Obteniendo eventos desde {timeMin} hasta {timeMax}')

    try:
        events_result = service.events().list(calendarId=calendar_id, timeMin=timeMin_iso,
                                              timeMax=timeMax_iso, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            logger.warning('No se encontraron eventos próximos.')
            return []

        eventos_formateados = []
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            eventos_formateados.append({
                'summary': event.get('summary', 'Sin título'),
                'start': start,
                'description': event.get('description', 'Sin descripción'),
                'location': event.get('location', 'Sin ubicación')
            })

        return eventos_formateados

    except HttpError as error:
        print(f'Ocurrió un error al obtener los eventos: {error}')
        return []


def get_calendar_event(date, service, calendar_id='primary'):
    """
    Obtiene los eventos del calendario para una fecha específica.

    Args:
        fecha (str): La fecha para la que se quieren obtener los eventos (formato 'YYYY-MM-DD').
        service: El servicio de la API de Google Calendar autorizado.
        calendar_id (str): El ID del calendario (por defecto, el calendario principal).

    Returns:
        str: Una cadena de múltiples líneas con los eventos encontrados, o un mensaje de error.
    """
    try:
        # Configurar las credenciales
        #creds = service_account.Credentials.from_service_account_file(credenciales_json, scopes=SCOPES)

        # Construir el servicio de la API de Calendar
        # service = build('calendar', 'v3', credentials=creds)
        
        # Convertir la fecha de entrada a un objeto datetime
        fecha_dt = datetime.datetime.strptime(date, '%Y-%m-%d').date()

        # Calcular el inicio y el fin del día
        hora_inicio = datetime.datetime.combine(fecha_dt, datetime.datetime.min.time())
        hora_fin = datetime.datetime.combine(fecha_dt + datetime.timedelta(days=1), datetime.datetime.min.time())

        # Formatear las fechas como RFC3339
        hora_inicio_iso = hora_inicio.isoformat() + 'Z'
        hora_fin_iso = hora_fin.isoformat() + 'Z'

        # Llamar a la API para obtener los eventos
        events_result = service.events().list(calendarId=calendar_id, timeMin=hora_inicio_iso,
                                              timeMax=hora_fin_iso, singleEvents=True,
                                              orderBy='startTime').execute()
        events = events_result.get('items', [])

        if not events:
            return 'No se encontraron eventos para la fecha especificada.'
        eventos_formateados = []
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            eventos_formateados.append({
                'summary': event.get('summary', 'Sin título'),
                'start': start,
                'description': event.get('description', 'Sin descripción'),
                'location': event.get('location', 'Sin ubicación')
            })

        return eventos_formateados

    except Exception as e:
        return f'Ocurrió un error: {e}'



In [152]:

# --------------------------------------------------------------
# Step 1: Definimos las estruturas de datos para enrutar al modelo
# --------------------------------------------------------------
conf_score_descrp = "Puntuación de confianza del 0 al 1"

# Crear lista de herramientas a  utilizar
class PrimaryRequestType(BaseModel):
    """ Llamada enrutadora a la LLM: Contiene qué tipo de solitud primaria está siendo realizada"""

    request_type: Literal["calendario", "extrac_inf","otro"] = Field(
        description="Tipo de solicitud primaria realizada"
    )
    confidence_score: float = Field(description=conf_score_descrp)
    description: str = Field(description="Descripción de la solicitud")

class CalendarRequestType(BaseModel):
    """Llamada enrutadora a la LLM: Contiene qué tipo de solicitud de calendario está siendo realizada"""

    request_type: Literal["nuevo_evento", "modif_evento", "extraer_mult_eventos","extraer_ind_eventos", "otro"] = Field(
        description="Tipo de solicitud de calendario realizada"
    )
    confidence_score: float = Field(description=conf_score_descrp)
    description: str = Field(description="Descripción de la solicitud")


class NewEventDetails(BaseModel):
    """Detalles para la creación de un nuevo evento"""

    name: str = Field(description="Nombre del evento")
    date: str = Field(description="Fecha y hora del evento. (ISO 8601)")
    duration_minutes: int = Field(description="Duración en minutos")
    participants: list[str] = Field(description="Lista de participantes")

class GetEventsDetails(BaseModel):
    """Detalles para la extracción de eventos"""
    cantidad: int = Field(description="Cantidad de eventos a extraer del calendario")

class Date(BaseModel):
    """Detalles para la extracción de eventos"""
    day: int = Field(description="Dia de la fecha")
    month: int = Field(description="Mes de la fecha")
    year: int = Field(description="Año de la fecha")

class Change(BaseModel):
    """Detalles a cambiar del evento existente"""

    field: str = Field(description="Campo a cambiar")
    new_value: str = Field(description="Nuevo valor a cambiar")


class ModifyEventDetails(BaseModel):
    """Detalles a modificar un evento existente"""

    event_identifier: str = Field(
        description="Descripción para identificar el evento existente"
    )
    changes: list[Change] = Field(description="Lista de cambios a realizar")
    participants_to_add: list[str] = Field(description="Nuevo participante a añadir")
    participants_to_remove: list[str] = Field(description="Participantes a remover")


class CalendarResponse(BaseModel):
    """Formato de respuesta final"""

    success: bool = Field(description="Si la operación tuvo éxito")
    message: str = Field(description="Mensaje de respuesta amistable al usuario")
    calendar_link: Optional[str] = Field(description="Link de calendario si aplica")


# --------------------------------------------------------------
# Step 2: Define el enrutamiento y las funciones de procesamiento 
# --------------------------------------------------------------

def route_primary_request(user_input: str) -> PrimaryRequestType:
    """ Enruta la llamada a la LLM para determinar el tipo de solicitud"""
    logger.info("Enrutando la solicitud primaria")
    
    client = genai.Client(api_key=API_KEY)
    system_instructions="Determina si la solicitud del usuario está relacionada con los calendarios o con extracción de información"
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {user_input}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : PrimaryRequestType
        }        
    )
    result : PrimaryRequestType = response.parsed
    logger.info(f"Solicitud enrutada como: {result.request_type} con una confianza de: {result.confidence_score}")
    return result
    

def route_calendar_request(user_input: str) -> CalendarRequestType:
    """Enruta la llamada a la LLM para determinar el tipo de solicitud de calendario"""
    logger.info("Enrutando solicitud de calendario")

    client = genai.Client(api_key=API_KEY)
    system_instructions="Determina si esta es una solicitud para crear un nuevo evento de calendario, modificar un evento existente o extraer eventos para múltiples días del calendario o extraer eventos para un día específicadp del calendario"
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {user_input}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : CalendarRequestType
        }        
    )
    result : CalendarRequestType = response.parsed
    logger.info(f"Solicitud enrutada como: {result.request_type} con una confianza de: {result.confidence_score}")
    return result



def handle_get_events(description: str) -> CalendarResponse:
    """Procesa un nuevo evento"""
    logger.info("Procesando una solicitud de extracción de eventos")

    client = genai.Client(api_key=API_KEY)
    system_instructions="Determina para cuántos días en el futuro el usuario desea extraer los eventos en su calendario. Si desea saber los eventos para el día actual, debes indicar un solo día."
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {description}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : GetEventsDetails
        }        
    )
    details : GetEventsDetails = response.parsed
    logger.info(f"Extrayendo los eventos para los proximos {details.cantidad} días")
    service = get_calendar_services()
    if service:
        events = get_calendar_events(service, details.cantidad)
    
    # Genera respuesta
    response = "Próximos Eventos:\n"
    for evento in events:
        response += (f"  - {evento['summary']}\n")
        response += (f"    Fecha/Hora: {evento['start']}\n")
        if evento['location']:
            response += (f"    Ubicación: {evento['location']}\n")
        if evento['description']:
            response += (f"    Descripción: {evento['description']}\n")
        response += ("-" * 20) + "\n"
    
    return response
    

def handle_get_event(description: str) -> CalendarResponse:
    """Procesa la extracción de un evento"""
    logger.info("Procesando una solicitud de extracción de eventos")
    today = datetime.datetime.today()
    client = genai.Client(api_key=API_KEY)
    system_instructions=f"La fecha actual es {today.day} del mes {today.month} del año {today.year}. En referencia a la fecha actual, extrae la fecha a la que se refiere el usuario. "
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {description}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : Date
        }        
    )
    details : Date = response.parsed
    logger.info(f"Extrayendo los eventos para: {details.year}-{details.month}-{details.day}")
    service = get_calendar_services()
    if service:
        events = get_calendar_event(f"{details.year}-{details.month}-{details.day}",service)
    # Genera respuesta
    response = "Próximos Eventos:\n"
    for evento in events:
        response += (f"  - {evento['summary']}\n")
        response += (f"    Fecha/Hora: {evento['start']}\n")
        if evento['location']:
            response += (f"    Ubicación: {evento['location']}\n")
        if evento['description']:
            response += (f"    Descripción: {evento['description']}\n")
        response += ("-" * 20) + "\n"
    
    return response
    


def handle_new_event(description: str) -> CalendarResponse:
    """Procesa un nuevo evento"""
    logger.info("Procesando una solicitud de un nuevo evento")

    client = genai.Client(api_key=API_KEY)
    system_instructions="Extrae los datos para la creación de un nuevo evento."
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {description}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : NewEventDetails
        }        
    )
    details : NewEventDetails = response.parsed
    logger.info(f"Nuevo evento: {details.model_dump_json(indent=2)}")

    # Genera respuesta
    return CalendarResponse(
        success=True,
        message=f"Creado un nuevo evento '{details.name}' para {details.date} con {', '.join(details.participants)}",
        calendar_link=f"calendar://new?event={details.name}",
    )


def handle_modify_event(description: str) -> CalendarResponse:
    """Procesa la modificación de un evento"""
    logger.info("Procesando una solicitud de modificación de evento")

    # Obtener detalles para modificación
    client = genai.Client(api_key=API_KEY)
    system_instructions="Extrae los datos para la modificación de un evento existente."
    response = client.models.generate_content(
        model=model,
        contents=f"{system_instructions}\n\nUser Query: {description}",
        config = { 
            "response_mime_type" : 'application/json',
            "response_schema" : ModifyEventDetails
        }        
    )
    details : ModifyEventDetails = response.parsed
    logger.info(f"Modificar Evento: {details.model_dump_json(indent=2)}")

    # Genera respuesta
    return CalendarResponse(
        success=True,
        message=f"Modificando evento '{details.event_identifier}' añadiendo a {', '.join(details.participants_to_add)} y eliminando a {', '.join(details.participants_to_remove)}",
        calendar_link=f"calendar://new?event={details.event_identifier}",
    )



def process_calendar_request(user_input: str) -> Optional[CalendarResponse]:
    """Main function implementing the routing workflow"""
    logger.info("Procesando solicitud de calendario")

    # Enrutar la solicitud
    route_result = route_calendar_request(user_input)

    # Evaluamos el grado de confianza del modelo
    if route_result.confidence_score < 0.7:
        logger.warning(f"Grado de confianza bajo: {route_result.confidence_score}")
        return None

    # Enrutar el handler apropiado
    if route_result.request_type == "nuevo_evento":
        return handle_new_event(route_result.description)
    elif route_result.request_type == "modif_evento":
        return handle_modify_event(route_result.description)
    elif route_result.request_type == "extraer_mult_eventos":
        return handle_get_events(route_result.description)
    elif route_result.request_type == "extraer_ind_eventos":
        return handle_get_event(route_result.description)
    else:
        logger.warning("Tipo de solicitud no soportada")
        return None


def process_primary_request(user_input: str):
    """Funcion principal para implementar el enrutamiento del trabajo"""
    logger.info("Procesando solicitud primaria")

    # Enrutar la solicitud
    route_result = route_primary_request(user_input)

    # Evaluamos el grado de confianza del modelo
    if route_result.confidence_score < 0.7:
        logger.warning(f"Grado de confianza bajo: {route_result.confidence_score}")
        return None
    
    # Enrutar el handler apropiado
    if route_result.request_type == "calendario":
        return process_calendar_request(route_result.description)
    elif route_result.request_type == "extrac_inf":
        logger.warning("Handler para funcion no implementado")
        return None
    else:
        logger.warning("Tipo de solicitud no soportada")
        return None


In [153]:
"""# --------------------------------------------------------------
# Step 3: Probar con nuevo evento
# --------------------------------------------------------------

new_event_input = "Programemos una reunión de equipo el próximo martes a las 2 p. m. con Alice y Bob."
result = process_primary_request(new_event_input)
if result:
    print(f"Respuesta: {result.message}")

# --------------------------------------------------------------
# Step 4: Probar con modificar un evento
# --------------------------------------------------------------

modify_event_input = (
    "¿Podrías trasladar la reunión del equipo con Alice y Bob al miércoles a las 3 p. m.?"
)
result = process_primary_request(modify_event_input)
if result:
    print(f"Respuesta: {result.message}")

# --------------------------------------------------------------
# Step 5: Probar con input inválido
# --------------------------------------------------------------

invalid_input = "¿Cómo está el tiempo hoy?"
result = process_primary_request(invalid_input)
if not result:
    print("Solicitud no reconocida como operación de calendario")
"""
# --------------------------------------------------------------
# Step 6: Probar con extraer eventos
# --------------------------------------------------------------


input = "¿Qué eventos tengo para el jueves?"

result = process_primary_request(input)

print(result)

if not result:
    print("Solicitud no reconocida como operación de calendario")


2025-03-10 15:39:50 - INFO - Procesando solicitud primaria
2025-03-10 15:39:50 - INFO - Enrutando la solicitud primaria
2025-03-10 15:39:50 - INFO - AFC is enabled with max remote calls: 10.
2025-03-10 15:39:52 - INFO - AFC remote call 1 is done.
2025-03-10 15:39:52 - INFO - Solicitud enrutada como: calendario con una confianza de: 0.95
2025-03-10 15:39:52 - INFO - Procesando solicitud de calendario
2025-03-10 15:39:52 - INFO - Enrutando solicitud de calendario
2025-03-10 15:39:52 - INFO - AFC is enabled with max remote calls: 10.
2025-03-10 15:39:53 - INFO - AFC remote call 1 is done.
2025-03-10 15:39:53 - INFO - Solicitud enrutada como: extraer_ind_eventos con una confianza de: 0.95
2025-03-10 15:39:53 - INFO - Procesando una solicitud de extracción de eventos
2025-03-10 15:39:53 - INFO - AFC is enabled with max remote calls: 10.
2025-03-10 15:39:55 - INFO - AFC remote call 1 is done.
2025-03-10 15:39:55 - INFO - Extrayendo los eventos para: 2025-3-13
2025-03-10 15:39:55 - INFO - fil

Próximos Eventos:
  - Python Star (12-13) Sección 3
    Fecha/Hora: 2025-03-13T16:00:00-04:00
    Ubicación: Sin ubicación
    Descripción: Curso Python Star de 12 a 13 años, sección 3
--------------------
  - Curso Python Pro-02-001
    Fecha/Hora: 2025-03-13T16:00:00-04:00
    Ubicación: Centro Empresarial AB, Av. Bolívar, Porlamar 6316, Nueva Esparta, Venezuela
    Descripción: Sin descripción
--------------------
  - Python Pro 1 (14-18) Sección 3
    Fecha/Hora: 2025-03-13T16:00:00-04:00
    Ubicación: Sin ubicación
    Descripción: Curso Python Pro 1, 14 a 18 años, sección 3.
--------------------

